<a href="https://colab.research.google.com/github/kennydavis120/HERCULES/blob/main/LatentWalk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from PIL import Image 
from torchvision import transforms as tfms  
import numpy as np 
import matplotlib.pyplot as plt 
import torch

In [2]:
pip install diffusers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# pip install transformers

In [4]:
from diffusers import AutoencoderKL
# from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler

In [5]:
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae", torch_dtype=torch.float16).to("cuda")
# model_id = "stabilityai/stable-diffusion-2"
# scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
# pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)
# vae = pipe.to("cuda")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


OSError: ignored

In [ ]:
def load_image(p):
    return Image.open(p).convert('RGB').resize((800,600))
def pil_to_latents(image):
    init_image = tfms.ToTensor()(image).unsqueeze(0) * 2.0 - 1.0   
    init_image = init_image.to(device="cuda", dtype=torch.float16)
    init_latent_dist = vae.encode(init_image).latent_dist.sample() * 0.18215     
    return init_latent_dist  
def latents_to_pil(latents):     
    latents = (1 / 0.18215) * latents     
    with torch.no_grad():         
        image = vae.decode(latents).sample     
    
    image = (image / 2 + 0.5).clamp(0, 1)     
    image = image.detach().cpu().permute(0, 2, 3, 1).numpy()      
    images = (image * 255).round().astype("uint8")     
    pil_images = [Image.fromarray(image) for image in images]        
    return pil_images

In [ ]:
image1 = "/HERCULES/image1.jpg"
image2 = "/HERCULES/image2.jpg"

In [ ]:
img_start = load_image(image1)
img_end = load_image(image2)

In [ ]:
pil1 = pil_to_latents(img_start)
pil2 = pil_to_latents(img_end)

In [ ]:
frame_num = 240
len_video = 4
my_pil_images = []
for i in range(frame_num):
    alpha = i/frame_num
    my_pil_images.append(latents_to_pil((1 - alpha) * pil1 + (alpha) * pil2))

In [ ]:
for i in range(frame_num):
  tmp = my_pil_images[i]
  tmp[0].save("/HERCULES/data/" + f"frame_{i:04d}.jpg")

In [ ]:
import moviepy.video.io.ImageSequenceClip
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

image_files = []

for img_number in range(1,240): 
    image_files.append("/HERCULES/data/" + f"frame_{img_number:04d}.jpg") 

fps = 30

clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)
clip.write_videofile('/HERCULES/my_new_video.mp4')